#Introduction

This notebook is a easy guide for training and testing of anomaly detection based on MVTec https://github.com/AdneneBoumessouer/MVTec-Anomaly-Detection.

###Link Google Drive

#Dataset 

### Directory Structure using your own dataset

To train with your own dataset, you need to have a comparable directory structure. For example:

In [ ]:
#data
  #├── class1
  # │   ├── test
  # │   │   ├── good
  # │   │   ├── defect
  # │   └── train
  # │       └── good
  # ├── class2
  # │   ├── test
  # │   │   ├── good
  # │   │   ├── defect
  # │   └── train
  # │       └── good
  # ...


#Training 

In [ ]:
# During training, the CAE trains exclusively on defect-free images and learns to reconstruct (predict) defect-free training samples.

# usage: train.py [-h] -d [-a] [-c] [-l] [-b] [-i]

# optional arguments:

# -h, --help show this help message and exit

# -d , --input-dir directory containing training images

# -a , --architecture architecture of the model to use for training: 'mvtecCAE', 'baselineCAE', 'inceptionCAE' or 'resnetCAE'

# -c , --color color mode for preprocessing images before training: 'rgb' or 'grayscale'

# -l , --loss loss function to use for training: 'mssim', 'ssim' or 'l2'

# -b , --batch batch size to use for training

# -i, --inspect generate inspection plots after training

# Example usage:

# python3 train.py -d mvtec/capsule -a mvtecCAE -b 8 -l ssim -c grayscale


In [ ]:
!python3 train.py -d data/can -a mvtecCAE -b 4 -l ssim -c grayscale


#Finetune 

In [ ]:
# This script used a subset of defect-free training images and a subset of both defect and defect-free test images to determine good values for minimum defect area and threshold pair of parameters that will be used during testing for classification and segmentation.

# usage: finetune.py [-h] -p [-m] [-t]

# optional arguments: -h, --help show this help message and exit

# -p , --path path to saved model

# -m , --method method for generating resmaps: 'ssim' or 'l2'

# -t , --dtype datatype for processing resmaps: 'float64' or 'uint8'

In [ ]:
!python3 finetune.py -p path/to/saved/model -m ssim -t float64

#Testing

In [ ]:
# This script classifies test images using the minimum defect area and threshold previously approximated at the finetuning step.

# usage: test.py [-h] -p [-s]

# optional arguments: -h, --help show this help message and exit

# -p , --path path to saved model

# -s, --save save segmented images

In [ ]:
!python3 test.py -p path/to/model

#Inference 

In [ ]:
from skimage.metrics import structural_similarity as ssim
import sys
import os
import argparse
from pathlib import Path
import shlex
import time
import json
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt     
from processing import utils
from processing import postprocessing
from processing.preprocessing import Preprocessor
from processing.preprocessing import get_preprocessing_function
from processing.postprocessing import label_images
from processing.utils import printProgressBar
from skimage.util import img_as_ubyte
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import glob
import cv2
import seaborn as sn
import seaborn as sns
import matplotlib.pyplot as plt    
import pandas as pd
import logging
import random
from sklearn.metrics import confusion_matrix
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


import keras
from autoencoder import losses
from autoencoder import metrics
from keras.models import load_model

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
def Test(args, th):
  model_path = args.path
  # load model and info
  model = load_model(model_path,
                      custom_objects={
                "LeakyReLU": keras.layers.LeakyReLU,
                "loss": losses.ssim_loss(1.0),
                "ssim": metrics.ssim_metric(1.0),
            },
            compile=True,
                      )
  # set parameters
  info = utils.get_model_info(model_path)
  input_directory = info["data"]["input_directory"]
  architecture = info["model"]["architecture"]
  loss = info["model"]["loss"]
  rescale = info["preprocessing"]["rescale"]
  shape = info["preprocessing"]["shape"]
  color_mode = info["preprocessing"]["color_mode"]
  vmin = info["preprocessing"]["vmin"]
  vmax = info["preprocessing"]["vmax"]
  nb_validation_images = info["data"]["nb_validation_images"]

  preprocessing_function = get_preprocessing_function(architecture)
  # # initialize preprocessor
  preprocessor = Preprocessor(
      input_directory=input_directory,
      rescale=rescale,
      shape=shape,
      color_mode=color_mode,
      preprocessing_function=preprocessing_function,
  )

  tst_img_list = [glob.glob(args.tst_pth + "/good/*.jpeg"), glob.glob(args.tst_pth + "/defect/*.jpeg")]
  tst_lbl_list = [list (np.ones(len(tst_img_list[0]))), list (np.zeros(len(tst_img_list[1])))]
  tst_prd_list = [list (np.ones(len(tst_img_list[0]))), list (np.zeros(len(tst_img_list[1])))]

  tst_img_list = tst_img_list[0] + tst_img_list[1]
  tst_lbl_list = tst_lbl_list[0] + tst_lbl_list[1]
  tst_prd_list = tst_prd_list[0] + tst_prd_list[1]
  
  for i,img_nm in enumerate(tst_img_list):
    img = preprocessor.get_test_image(img_nm)
    imgs_test_pred = model.predict(img)
    
    ssim_l = 0
    
    for pat in range(20):
        x = random.randint(0, 256)
        y = random.randint(0, 256)
        imgs_test_pred_pth = imgs_test_pred.reshape((512,512))
        imgs_test_pred_pth = imgs_test_pred_pth[y: y + 256, x: x + 256]
        
        img_pth = img.reshape((512,512))
        img_pth = img_pth[y: y + 256, x: x + 256]
        
        
        
        
        img_pth = img_pth.reshape(256,256)
        imgs_test_pred_pth = imgs_test_pred_pth.reshape(256,256)
        
        ssim_ = ssim(img_pth, imgs_test_pred_pth, data_range=imgs_test_pred_pth.max() - imgs_test_pred_pth.min(), full = True)
        
        val = 1- ssim_[0]
        ssim_l+=val
    ssim_l = ssim_l/20
        
    
    print("cnt_1", f" :{ssim_l}",tst_img_list[i], f" count: {ssim_l}")

    
    if ssim_l > th:
      tst_prd_list[i] = 0
    else:
      tst_prd_list[i] = 1

# confusion matrix
  matrix = confusion_matrix(tst_lbl_list,tst_prd_list, labels=[1,0])
  print('Classification report : \n',matrix)
  return matrix

In [ ]:
parser = argparse.ArgumentParser() 
parser.add_argument("-p",
    "--path", type=str, default= None ,required=True, metavar="", help="path to saved model"
)
parser.add_argument(
    "-i", "--img_pth", type=str, default = 'None', help="save segmented images",
)

parser.add_argument(
     "--tst_pth", type=str, default = None, help="path to test folder",
)

notebook_args = f"""
--path "./saved_models/data/can/mvtecCAE/ssim/7 gaussian_worked_v6/ad_model.h5"
--img_pth "./data/can/test/good/20210914-180510.138.jpeg"
--tst_pth "./data/can/test_"
"""
args = parser.parse_args(shlex.split(notebook_args))

In [ ]:
matr = Test(args, 0.035)

In [ ]:
ax= plt.subplot()
sns.heatmap(matr, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation
# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['good', 'defect']); ax.yaxis.set_ticklabels(['good', 'defect']);

In [ ]:
def inference(args, th):
    # parse arguments
    model_path = args.path
    # ============= LOAD MODEL AND PREPROCESSING CONFIGURATION ================
    # load model and info
    model = load_model(model_path,
                      custom_objects={
                "LeakyReLU": keras.layers.LeakyReLU,
                "loss": losses.ssim_loss(1.0),
                "ssim": metrics.ssim_metric(1.0),
            },
            compile=True,
                      )
  # set parameters
    info = utils.get_model_info(model_path)
    # set parameters
    input_directory = info["data"]["input_directory"]
    architecture = info["model"]["architecture"]
    loss = info["model"]["loss"]
    rescale = info["preprocessing"]["rescale"]
    shape = info["preprocessing"]["shape"]
    color_mode = info["preprocessing"]["color_mode"]
    vmin = info["preprocessing"]["vmin"]
    vmax = info["preprocessing"]["vmax"]
    nb_validation_images = info["data"]["nb_validation_images"]

    

    #     # get the correct preprocessing function
    preprocessing_function = get_preprocessing_function(architecture)

    # # initialize preprocessor
    preprocessor = Preprocessor(
        input_directory=input_directory,
        rescale=rescale,
        shape=shape,
        color_mode=color_mode,
        preprocessing_function=preprocessing_function,
    )

    # get test image
    img = preprocessor.get_test_image(args.img_pth)
    
    flp_img = np.flip(img, axis = 2)
    plt.imshow(flp_img.reshape(512,512), cmap="gray")
    plt.show()
#     x = random.randint(0, 256)
#     y = random.randint(0, 256)
#     img = img.reshape((512,512))
#     img = img[y: y + 256, x: x + 256]
#     img = cv2.resize(img, (512,512), interpolation = cv2.INTER_AREA)
#     img = img.reshape(1,512,512,1)
        
    
    
    plt.imshow(img.reshape(512,512), cmap='gray')
    plt.show()
    #predict on test image
    start_time = time.time()

    imgs_test_pred = model.predict(img)
    imgs_test_pred_flp = model.predict(flp_img)
    
    img_t = tf.convert_to_tensor(img, np.float32)
    img_pred__t = tf.convert_to_tensor(imgs_test_pred, np.float32)
    ssim_loss = 1 - tf.image.ssim(img_t, img_pred__t, 1)
    print(ssim_loss)
    
    plt.imshow(imgs_test_pred.reshape(512,512), cmap='gray')
    plt.show()

#     ssim_ = ssim(img.reshape((512,512)), imgs_test_pred.reshape((512,512)), data_range=img.max() - img.min(), full = True)
# #     ssim_ = np.asarray(ssim_)
#     ssim_ = ssim_[1]
#     ssim_ = 1 - ((ssim_ - np.min(ssim_))/np.ptp(ssim_))
#     print(ssim_.shape)
#     plt.imshow(ssim_.reshape(512,512), cmap='gray')
#     plt.show()
    
#     ssim_ = -1 * ssim_ * np.log(ssim_)
#     plt.imshow(ssim_)
#     plt.show()
#     arr = np.where(ssim_ > 0.36, 1.0,0)
    
    
    map_ = np.sqrt((imgs_test_pred - img)**2)
    map_ = ((map_ - np.min(map_))/np.ptp(map_))
    
    plt.imshow(map_.reshape(512,512), cmap="gray")
    plt.show()
    
#     ent = -1 * map_ * np.log(map_)
    end_map = np.where(map_ > 0.6, 1.0,0)
#     plt.imshow(ent.reshape((512,512)))
#     plt.show()
    
    
#     map_ = glb_mask * arr
#     cnt_n = np.count_nonzero(map_)
#     print(cnt_n)
    
#     plt.imshow(arr.reshape(512,512), cmap="gray")
#     plt.show()
    
    plt.imshow(end_map.reshape(512,512), cmap="gray")
    plt.show()
    
    print("-----------------")
    map_ = np.sqrt((imgs_test_pred_flp - flp_img)**2)
    map_ = ((map_ - np.min(map_))/np.ptp(map_))
    
    plt.imshow(map_.reshape(512,512), cmap="gray")
    plt.show()
    
#     ent = -1 * map_ * np.log(map_)
    end_map = np.where(map_ > 0.6, 1.0,0)
    
    plt.imshow(end_map.reshape(512,512), cmap="gray")
    plt.show()


    

In [ ]:
img_shp = 512
cnt = int(img_shp/2)
glb_mask = cv2.circle(np.zeros((img_shp, img_shp)), (cnt-10, cnt-10), cnt , (1,1,1), -1) 
glb_mask = np.array(glb_mask, dtype = 'uint8')

In [ ]:
inference(args, 0.2)

In [ ]:
classification_algo(args)